In [ ]:
import mpp_classifiers as mpp
import load_ship_data as lsd
import numpy as np
import time
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold

generate labels for classifier fusion

In [17]:

data_path="data/shipsnet.json"
train,test,valid=lsd.load_data_train_test_split(data_path)
def conf(pred,y):
    T0=sum([1 if x==y and y==0 else 0 for (x,y) in zip(pred,y)])
    T1=sum([1 if x==y and y==1 else 0 for (x,y) in zip(pred,y)])
    F0=sum([1 if x!=y and y==0 else 0 for (x,y) in zip(pred,y)])
    F1=sum([1 if x!=y and y==1 else 0 for (x,y) in zip(pred,y)])
    return np.array([[T0,F0],[F1,T1]])

Training Set Data Length:  2800   Label Length:  2800
TestingSet Set Data Length:  600  Label Length:  600
Validation Set Data Length:  600  Label Length:  600


Setup data splits and priors

In [20]:
ship_prob=np.sum(train[1])*1.0/train[1].shape[0]
p=np.array([1-ship_prob,ship_prob])
print(p)
Xtrain=train[0]
ytrain=train[1]
Xtest=test[0]
ytest=test[1]
Xvalid=valid[0]
yvalid=valid[1]
Xtrain=Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1]*Xtrain.shape[2]*Xtrain.shape[3])
Xtest=Xtest.reshape(Xtest.shape[0],Xtest.shape[1]*Xtest.shape[2]*Xtest.shape[3])
Xvalid=Xvalid.reshape(Xvalid.shape[0],Xvalid.shape[1]*Xvalid.shape[2]*Xvalid.shape[3])
print(Xtrain.shape)
print(Xtest.shape)
print(Xvalid.shape)


Xtrain=np.concatenate((Xtrain,Xvalid))
ytrain=np.concatenate((ytrain,yvalid))
print(Xtrain.shape)

[0.74857143 0.25142857]
(2800, 19200)
(600, 19200)
(600, 19200)
(3400, 19200)


PCA =200 dimensions kept

In [21]:
pca2 = PCA(n_components=200)
pca2.fit(Xtrain)
pca2_Xtrain=pca2.transform(Xtrain)
pca2_Xtest=pca2.transform(Xtest)
explained=np.sum(pca2.explained_variance_ratio_)
print(explained)

0.9396689324818621


Case 1. PCA=200

In [30]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtrain, 1, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytrain, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytrain)
print(ar)
np.save("ground_truth.npy",ytrain)
np.save("case_1.npy",y_model)

Overall accuracy = 0.7567647058823529;
Classwise accuracy = [1.         0.01076555];
The learning process takes 0.058840274810791016 seconds.
[[2564    0]
 [ 827    9]]


Case 2. PCA=200

In [31]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtrain, 2, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytrain, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytrain)
print(ar)
np.save("case_2.npy",y_model)

Overall accuracy = 0.9338235294117647;
Classwise accuracy = [0.96723869 0.83133971];
The learning process takes 2.4584543704986572 seconds.
[[2480   84]
 [ 141  695]]


Case 3. PCA=200

In [32]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtrain, 3, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytrain, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytrain)
print(ar)
np.save("case_3.npy",y_model)

Overall accuracy = 0.8394117647058823;
Classwise accuracy = [0.79290172 0.98205742];
The learning process takes 4.682505369186401 seconds.
[[2033  531]
 [  15  821]]


In [33]:
print(ytrain.shape)
print(y_model.shape)

(3400,)
(3400,)
